## FORTUNE 500 TABLE

In [1]:
SELECT * FROM 'fortune_New.csv';

,rank,title,name,ticker,url,hq,sector,industry,employees,revenues,revenues_change,profits,profits_change,assets,equity
0,1,Walmart,"Wal-Mart Stores, Inc.",WMT,http://www.walmart.com,"Bentonville, AR",Retailing,General Merchandisers,2300000,485873.00,0.8,13643.00,-7.20,198825.0,77798.0
1,2,Berkshire Hathaway,Berkshire Hathaway Inc.,BRKA,http://www.berkshirehathaway.com,"Omaha, NE",Financials,Insurance: Property and Casualty (Stock),367700,223604.00,6.1,24074.00,0.00,620854.0,283001.0
2,3,Apple,"Apple, Inc.",AAPL,http://www.apple.com,"Cupertino, CA",Technology,"Computers, Office Equipment",116000,215639.00,-7.7,45687.00,-14.40,321686.0,128249.0
3,4,Exxon Mobil,Exxon Mobil Corporation,XOM,http://www.exxonmobil.com,"Irving, TX",Energy,Petroleum Refining,72700,205004.00,-16.7,7840.00,-51.50,330314.0,167325.0
4,5,McKesson,McKesson Corporation,MCK,http://www.mckesson.com,"San Francisco, CA",Wholesalers,Wholesalers: Health Care,68000,192487.00,6.2,2258.00,53.00,56563.0,8924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Michaels Cos.,"The Michaels Companies, Inc.",MIK,http://www.michaels.com,"Irving, TX",Retailing,Specialty Retailers: Other,31000,5197.00,5.8,378.20,4.20,2148.0,-1698.0
496,497,Toll Brothers,"Toll Brothers, Inc.",TOL,http://www.tollbrothers.com,"Horsham, PA",Engineering & Construction,Homebuilders,4200,5170.00,23.9,382.10,5.20,9737.0,4229.0
497,498,Yahoo,Yahoo! Inc.,YHOO,http://www.yahoo.com,"Sunnyvale, CA",Technology,Internet Services and Retailing,8500,5169.00,4.0,-214.30,NA,48083.0,31049.0
498,499,Vistra Energy,Vistra Energy Corp.,VST,http://www.vistraenergy.com,"Dallas, TX",Energy,Energy,4431,5164.00,4.0,NA,NA,15167.0,6597.0


## STOCK EXCHANGE TABLE

In [17]:
SELECT * FROM 'stackexchange.csv';

,tag,date,question_count,question_pct,unanswered_count,unanswered_pct
0,paypal,2018-09-25 00:00:00+00:00,18050.0,0.001090,8402.0,0.001750
1,amazon-elb,2018-09-25 00:00:00+00:00,1452.0,0.000088,561.0,0.000117
2,amazon-mws,2018-09-25 00:00:00+00:00,706.0,0.000043,278.0,0.000058
3,amazon-swf,2018-09-25 00:00:00+00:00,232.0,0.000014,77.0,0.000016
4,amazon-sns,2018-09-25 00:00:00+00:00,1400.0,0.000085,601.0,0.000125
...,...,...,...,...,...,...
45233,azure,2016-01-01 00:00:00+00:00,25859.0,0.002400,8592.0,0.002980
45234,android,2016-01-01 00:00:00+00:00,770606.0,0.071400,280003.0,0.097000
45235,paypal,2016-01-01 00:00:00+00:00,13296.0,0.001230,5933.0,0.002050
45236,stripe-payments,2016-01-01 00:00:00+00:00,1736.0,0.000161,700.0,0.000242


## Compute the correlation between revenues and other financial variables with the corr() function


SELECT corr(revenues,profits) AS rev_profits ,
       
       corr(revenues,assets) AS rev_assets ,
	   
       corr(revenues,equity) AS rev_equity 
	   
FROM fortune500;

In [14]:
--- Compute the correlation between revenues and other financial variables with the corr() function

SELECT corr(revenues,profits) AS rev_profits ,
       corr(revenues,assets) AS rev_assets ,
	   corr(revenues,equity) AS rev_equity 
	   
FROM fortune500;

,rev_profits,rev_assets,rev_equity
0,0.599994,0.3295,0.546571


## Compute the mean and median assets of Fortune 500 companies by sector


SELECT sector,
       
       avg(assets) AS mean,
       
       percentile_disc(0.5) within group (order by assets) AS median
  
  FROM fortune500
 
 GROUP BY sector

 ORDER BY sector;


In [15]:
-- What groups are you computing statistics by?
SELECT sector,
       -- Select the mean of assets with the avg function
       avg(assets) AS mean,
       -- Select the median
       percentile_disc(0.5) within group (order by assets) AS median
  FROM fortune500
 -- Computing statistics for each what?
 GROUP BY sector
 -- Order results by a value of interest
 ORDER BY sector;

,sector,mean,median
0,Aerospace & Defense,31897.666667,20038.0
1,Apparel,11064.800000,9739.0
2,Business Services,19626.100000,12485.0
3,Chemicals,20151.214286,15769.0
4,Energy,48756.210526,36119.0
5,Engineering & Construction,8199.230769,8709.0
6,Financials,319245.095238,123449.0
7,Food & Drug Stores,24630.714286,17464.0
8,"Food, Beverages & Tobacco",29059.750000,15984.0
9,Health Care,42078.894737,25396.0


## Find the Fortune 500 companies that have profits in the top 20% for their sector (compared to other Fortune 500 companies).



DROP TABLE IF EXISTS profit80;

CREATE TEMP TABLE profit80 AS
  
  SELECT sector, 
         
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
   
   FROM fortune500 
   
   GROUP BY sector;



SELECT title, fortune500.sector, profits,
        
        profits/pct80 AS ratio

  FROM fortune500 
       
       LEFT JOIN profit80

       ON profit80.sector = fortune500.sector

 WHERE profits > pct80;

In [22]:
-- Code from previous step
DROP TABLE IF EXISTS profit80;

CREATE TEMP TABLE profit80 AS
  SELECT sector, 
         percentile_disc(0.8) WITHIN GROUP (ORDER BY profits) AS pct80
    FROM fortune500 
   GROUP BY sector;

-- Select columns, aliasing as needed
SELECT title, fortune500.sector, profits,
        profits/pct80 AS ratio
-- What tables do you need to join?  
  FROM fortune500 
       LEFT JOIN profit80
-- How are the tables joined?
       ON profit80.sector = fortune500.sector
-- What rows do you want to select?
 WHERE profits > pct80;

,title,sector,profits,ratio
0,Walmart,Retailing,13643.0,11.109935
1,Berkshire Hathaway,Financials,24074.0,7.987392
2,Apple,Technology,45687.0,6.287779
3,Exxon Mobil,Energy,7840.0,5.980168
4,McKesson,Wholesalers,2258.0,3.726688
...,...,...,...,...
85,AutoZone,Retailing,1241.0,1.010586
86,Sempra Energy,Energy,1370.0,1.045004
87,Weyerhaeuser,Materials,1027.0,2.070565
88,PPL,Energy,1902.0,1.450801


## **Find out how many questions had each tag on the first date for which data for the tag is available, as well as how many questions had the tag on the last day. Also, compute the difference between these two values.**


DROP TABLE IF EXISTS startdates;

CREATE TEMP TABLE startdates AS

SELECT tag, min(date) AS mindate

  FROM stackoverflow
  
 GROUP BY tag;
 

SELECT startdates.tag, 

       mindate, 
       
	   so_min.question_count AS min_date_question_count,
       
       so_max.question_count AS max_date_question_count,
     
       so_max.question_count - so_min.question_count AS change
       
  FROM startdates
      
       INNER JOIN stackoverflow AS so_min
          
          ON startdates.tag = so_min.tag
          
         AND startdates.mindate = so_min.date
      
       INNER JOIN stackoverflow AS so_max
       	 
          ON startdates.tag = so_max.tag
          
         AND so_max.date = '2018-09-25';



In [21]:
-- To clear table if it already exists
DROP TABLE IF EXISTS startdates;

CREATE TEMP TABLE startdates AS
SELECT tag, min(date) AS mindate
  FROM stackoverflow
 GROUP BY tag;
 
-- Select tag (Remember the table name!) and mindate
SELECT startdates.tag, 
       mindate, 
       -- Select question count on the min and max days
	   so_min.question_count AS min_date_question_count,
       so_max.question_count AS max_date_question_count,
       -- Compute the change in question_count (max- min)
       so_max.question_count - so_min.question_count AS change
  FROM startdates
       -- Join startdates to stackoverflow with alias so_min
       INNER JOIN stackoverflow AS so_min
          -- What needs to match between tables?
          ON startdates.tag = so_min.tag
         AND startdates.mindate = so_min.date
       -- Join to stackoverflow again with alias so_max
       INNER JOIN stackoverflow AS so_max
       	  -- Again, what needs to match between tables?
          ON startdates.tag = so_max.tag
         AND so_max.date = '2018-09-25';

,tag,mindate,min_date_question_count,max_date_question_count,change
0,applepay,2017-03-18 00:00:00+00:00,222.0,357.0,135.0
1,applepayjs,2017-03-18 00:00:00+00:00,11.0,30.0,19.0
2,android-pay,2017-03-17 00:00:00+00:00,444.0,490.0,46.0
3,amazon-kinesis,2016-09-01 00:00:00+00:00,259.0,766.0,507.0
4,amazon-sns,2016-09-01 00:00:00+00:00,690.0,1400.0,710.0
5,amazon-emr,2016-09-01 00:00:00+00:00,557.0,3046.0,2489.0
6,amazon-swf,2016-09-01 00:00:00+00:00,167.0,232.0,65.0
7,amazon-ecs,2016-09-01 00:00:00+00:00,145.0,1074.0,929.0
8,amazon-rds,2016-09-01 00:00:00+00:00,1156.0,2537.0,1381.0
9,amazon-ses,2016-09-01 00:00:00+00:00,481.0,934.0,453.0


## Compute the correlations between each pair of profits, profits_change, and revenues_change from the Fortune 500 data


DROP TABLE IF EXISTS correlations;

CREATE TEMP TABLE correlations AS

SELECT 'profits'::varchar AS measure,

       corr(profits, profits) AS profits,
       
       corr(profits, profits_change) AS profits_change,
       
       corr(profits, revenues_change) AS revenues_change
       
  FROM fortune500;

INSERT INTO correlations

SELECT 'profits_change'::varchar AS measure,

       corr(profits_change, profits) AS profits,
       
       corr(profits_change, profits_change) AS profits_change,
       
       corr(profits_change, revenues_change) AS revenues_change
       
  FROM fortune500;

INSERT INTO correlations

SELECT 'revenues_change'::varchar AS measure,

       corr(revenues_change, profits) AS profits,
       
       corr(revenues_change, profits_change) AS profits_change,
       
       corr(revenues_change, revenues_change) AS revenues_change
       
  FROM fortune500;


SELECT measure, 

       round(profits::numeric, 2) AS profits,
       
       round(profits_change::numeric, 2) AS profits_change,
       
       round(revenues_change::numeric, 2) AS revenues_change
       
  FROM correlations;



In [20]:
DROP TABLE IF EXISTS correlations;

CREATE TEMP TABLE correlations AS
SELECT 'profits'::varchar AS measure,
       corr(profits, profits) AS profits,
       corr(profits, profits_change) AS profits_change,
       corr(profits, revenues_change) AS revenues_change
  FROM fortune500;

INSERT INTO correlations
SELECT 'profits_change'::varchar AS measure,
       corr(profits_change, profits) AS profits,
       corr(profits_change, profits_change) AS profits_change,
       corr(profits_change, revenues_change) AS revenues_change
  FROM fortune500;

INSERT INTO correlations
SELECT 'revenues_change'::varchar AS measure,
       corr(revenues_change, profits) AS profits,
       corr(revenues_change, profits_change) AS profits_change,
       corr(revenues_change, revenues_change) AS revenues_change
  FROM fortune500;

-- Select each column, rounding the correlations
SELECT measure, 
       round(profits::numeric, 2) AS profits,
       round(profits_change::numeric, 2) AS profits_change,
       round(revenues_change::numeric, 2) AS revenues_change
  FROM correlations;

,measure,profits,profits_change,revenues_change
0,profits,1.00,0.02,0.02
1,profits_change,0.02,1.00,-0.09
2,revenues_change,0.02,-0.09,1.00
